In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
from numpy import radians, sin, cos, arcsin, sqrt
def dist(a, b):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    x1, y1, x2, y2 = map(radians, [a[0], a[1], b[0], b[1]])
    dx = x2 - x1
    dy = y2 - y1
    a = sin(dy/2)**2 + cos(y1) * cos(y2) * sin(dx/2)**2
    c = 2 * arcsin(sqrt(a))
    # Earth`s radius in kilometers
    km = 6372.795 * c
    return km

In [ ]:
plt.style.use('classic')   # try another styles: 'classic'
plt.rcParams['figure.figsize'] = [15, 15]

srcData = "G:/1_Data1/sea/AllSea.txt"
destData = "G:/1_Data1/sea/AllSeaGisOut.txt"
obrisData = "G:/1_Data1/obris/obris.dat"
batimetrFile = "G:/1_Data1/batimetria/b2.txt"
# srcData = "~/methaneSea.dat

In [ ]:
df = pd.read_csv(srcData,
                 header=0,
                 # skip_blank_lines=True,
                 skipinitialspace=True,
                 na_values='--',
                 skiprows=[1,2,3,4],
                 sep='\t',
                 decimal=',',
                 # nrows=100000,
                 parse_dates=['DateTime'],
                 dayfirst=True
                 )

In [ ]:
df.shape

In [ ]:
df.dropna(axis='index', subset=['Longitude', 'Latitude', 'cCH4Wtr'], inplace=True)
df.shape

In [ ]:
df['year'] = df['DateTime'].dt.year

In [ ]:
df.reset_index(drop=True, inplace=True)
df

In [ ]:
for yr in range(2013, 2024):
    t = df.loc[(df['year'] == yr) & df['cCH4Wtr']].shape
    print(f'{yr}: {t[0]}')

In [ ]:
obris = pd.read_csv(obrisData,
                    header=0,
                    na_values='--',
                    sep='\t',
                    decimal=',',
                    )
obris.head(3)

In [ ]:
batlegend = pd.read_csv(batimetrFile,
                         nrows=6,
                         decimal='.',
                         sep='\s+',
                         index_col=0,
                         header=None,
                        )
batlegend= batlegend.T
na_val = batlegend['nodata_value']

In [ ]:
batimetria = pd.read_csv(batimetrFile,
                         skiprows=6,
                         decimal='.',
                         sep='\s+',
                         na_values=na_val,
                         index_col=False,
                         header=None,
                        )

# batimetria.dtypes\
xllcorner = batlegend.loc[1, 'xllcorner']
yllcorner = batlegend.loc[1, 'yllcorner']
cellsize = batlegend.loc[1, 'cellsize']

In [ ]:
batimetria

In [ ]:
plt.imshow(batimetria)

In [ ]:
#df['depth'] = batimetria.iloc[((df['Latitude'] - yllcorner) / cellsize).astype('int32'),((df['Longitude'] - xllcorner) / cellsize).astype('int32')]
yllcorner_m = yllcorner + batimetria.shape[0] * cellsize
yllcorner_m

In [ ]:
df['batiy'] = ((yllcorner_m - df['Latitude']) / cellsize).astype('int32')
df['batix'] = ((df['Longitude'] - xllcorner) / cellsize).astype('int32')
df

In [ ]:
#df = (pd.merge(df, batimetria.stack().rename("depth"), left_on=["batiy", "batix"], right_index=True, how="left"))
df['depth'] = batimetria.to_numpy()[df['batiy'], df['batix']]
df

In [ ]:
plt.scatter(df['Longitude'], df['Latitude'], s=30, c=df['depth'], marker='o', linewidth=0, alpha=0.1)
plt.plot(obris['long'], obris['lat'], 'b-')

In [ ]:
longStep = 0.01
latStep = 0.005

df['long'] = (df['Longitude']/longStep).round()*longStep
df['lat'] = (df['Latitude']/latStep).round()*latStep
df['year'] = df['DateTime'].dt.year

In [ ]:
#func_list = ['mean', 'count']
func_list = ['mean']
cols_to_calc = {'pCO2Wtr': func_list,
                'pCH4Wtr': func_list,
                'cCO2Wtr': func_list,
                'cCH4Wtr': func_list,
                'dpCO2': func_list,
                'dpCH4': func_list,
                'pCO2Air': func_list,
                'pCH4Air': func_list,
                'depth': ['mean']}
res = df.groupby(['year', 'long', 'lat'], as_index=False).agg(cols_to_calc)
res

In [ ]:
for yr in range(2013, 2024):
    t = res.loc[res['year'] == yr].shape
    print(f'{yr}: {t[0]}')

In [ ]:
func_list = ['mean', 'std', 'count']

cols_to_calc = {('cCH4Wtr', 'mean'): func_list,
                ('depth', 'mean'): func_list,
               }
res = res.groupby(['long', 'lat'], as_index=False).agg(cols_to_calc)
res

In [ ]:
res.to_csv(destData,
           sep='\t',
           )

In [ ]:
res

In [ ]:
plt.style.available

In [ ]:
%matplotlib inline
plt.style.use('bmh')   # try another styles: 'classic'
valMax = res[('cCH4Wtr','mean')].max

fig = plt.figure()

# wether this or those
plt.scatter(res['long'], res['lat'], c=res[('cCH4Wtr', 'mean', 'mean')], s=10, marker='s', linewidth=0)
plt.plot(obris['long'], obris['lat'], 'b-')

fig.savefig('gis.png')